In [1]:
from analysis_utils import get_run_data, process_run_data, aggregate_metrics, METRIC_NAMES

In [8]:
data, config = get_run_data("eli-carrami/Cprt-Paper-Exp-1")
out = []
best_basis = "biochem/val_localization_f1"
use_last = False
for d, c in zip(data, config):
    llm = c['model']['value']['language_model']
    if 'gpt' in llm:
        h = d.iloc[-1].copy()
    elif use_last:
        d = d[d.epoch == 0]
        h = d.iloc[-1].copy()
    else:
        h = d.iloc[d[best_basis].idxmax()].copy()
    h['protein_layer_to_use'] = c['model']['value']['protein_layer_to_use']
    h['protein_layer_to_use'] = -1 if h['protein_layer_to_use'] == 12 else h['protein_layer_to_use']
    h['esm'] = c['model']['value']['protein_model']
    h['llm'] = llm
    h['strategy'] = c['model']['value']['multimodal_strategy']
    h['seed'] = c['seed']['value']
    out.append(h)

clear-yogurt-10 crisp-resonance-64 devoted-wildflower-61 summer-leaf-52 rural-glade-24 scarlet-totem-23 wise-bush-22 misunderstood-waterfall-21 fanciful-snowflake-36 vague-silence-25 iconic-sea-22 elated-frog-8 pretty-butterfly-6 efficient-frost-5 volcanic-lion-4 chocolate-pyramid-3 young-salad-2 autumn-wildflower-1 

In [19]:
model_order = ["esm2_t36_3B_UR50D", "esm2_t33_650M_UR50D", "esm2_t12_35M_UR50D", "esm2_t6_8M_UR50D"]
var = 'esm'
fltr = [('strategy', "soft-prompt"), ('llm', "gpt2-medium")]
ordering = (var, model_order)
df = process_run_data(out, fltr, ordering)

In [20]:
df

,biochem/val_is_real_f1,biochem/val_is_enzyme_hard_f1,biochem/val_kingdom_f1,biochem/val_localization_f1,biochem/val_cofactor,biochem/val_is_fake_f1,biochem/val_mw_error,metrics/val_perplexity,metrics/val_rouge1_fmeasure,metrics/val_rouge1_precision,...,biochem/val_in_membrane_f1,biochem/val_in_nucleus_f1,biochem/val_in_mitochondria_f1,epoch,protein_layer_to_use,esm,llm,strategy,seed,avg_binary_loc_f1
13,0.990010,0.868263,0.838981,0.769025,0.222222,0.178689,0.216861,3.185420,0.598898,0.604275,...,0.205802,0.183664,0.147578,4.0,-1.0,esm2_t36_3B_UR50D,gpt2-medium,soft-prompt,0,0.179015
14,0.984027,0.840242,0.870506,0.709540,0.305556,0.009976,0.055413,3.080299,0.582835,0.573277,...,0.396047,0.341236,0.388061,4.0,-1.0,esm2_t33_650M_UR50D,gpt2-medium,soft-prompt,0,0.375115
4,0.988138,0.863928,0.896312,0.722203,0.394231,0.132075,0.040675,3.016529,0.595260,0.593815,...,0.197922,0.132412,0.069166,4.0,-1.0,esm2_t33_650M_UR50D,gpt2-medium,soft-prompt,7,0.133167
5,0.990029,0.878154,0.922889,0.654014,0.260274,0.169657,0.103959,3.068007,0.617433,0.643394,...,0.225380,0.123733,0.161129,4.0,-1.0,esm2_t33_650M_UR50D,gpt2-medium,soft-prompt,42,0.170081
12,0.960076,0.836139,0.668738,0.680844,0.222222,0.118233,0.224499,3.288699,0.597508,0.609539,...,0.189411,0.170015,0.077547,4.0,-1.0,esm2_t6_8M_UR50D,gpt2-medium,soft-prompt,0,0.145657
7,0.978254,0.839557,0.674763,0.586473,0.211538,0.223926,0.143106,3.197092,0.599283,0.606511,...,0.374451,0.372283,0.379022,4.0,-1.0,esm2_t6_8M_UR50D,gpt2-medium,soft-prompt,7,0.375252
6,0.973996,0.832070,0.769413,0.674791,0.232877,0.034633,0.066999,3.279284,0.611650,0.639912,...,0.326909,0.213632,0.228344,4.0,-1.0,esm2_t6_8M_UR50D,gpt2-medium,soft-prompt,42,0.256295


In [21]:
metrics_names = {k:v for k, v in METRIC_NAMES.items() if 'rouge' not in k}
agg_df = aggregate_metrics(df, group_by=var)
agg_df = agg_df[[col for col in metrics_names]]

for col, name in metrics_names.items():
    agg_df[name] = round(agg_df[(col, 'mean')], 2).astype(str) + " (" + round(agg_df[(col, 'std')], 2).astype(str) + ")"
    agg_df.drop([(col, 'mean'), (col, 'std')], axis=1, inplace=True)

agg_df.to_clipboard()
agg_df

/Users/elicarrami/PycharmProjects/cprt/cprt/analysis/analysis_utils.py:107: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return df.groupby(group_by).agg(['mean', 'std'])


,is_real F1,is_enzyme F1,kingdom F1,localization F1,cofactor Recall,binary localization\naverage F1,is_fake F1,MW MALE,perplexity,in_membrane F1,in_nucleus F1,in_mitochondria F1
,,,,,,,,,,,,
esm,,,,,,,,,,,,
esm2_t36_3B_UR50D,0.99 (nan),0.87 (nan),0.84 (nan),0.77 (nan),0.22 (nan),0.18 (nan),0.18 (nan),0.22 (nan),3.19 (nan),0.21 (nan),0.18 (nan),0.15 (nan)
esm2_t33_650M_UR50D,0.99 (0.0),0.86 (0.02),0.9 (0.03),0.7 (0.04),0.32 (0.07),0.23 (0.13),0.1 (0.08),0.07 (0.03),3.05 (0.03),0.27 (0.11),0.2 (0.12),0.21 (0.16)
esm2_t12_35M_UR50D,nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan)
esm2_t6_8M_UR50D,0.97 (0.01),0.84 (0.0),0.7 (0.06),0.65 (0.05),0.22 (0.01),0.26 (0.11),0.13 (0.09),0.14 (0.08),3.26 (0.05),0.3 (0.1),0.25 (0.11),0.23 (0.15)
